In [ ]:
dbutils.library.restartPython()

# Import de Funções e Libs

As bibliotecas e funções utilizadas ao longo do código devem ser adicionadas aqui

In [ ]:
import numpy as np
import pandas as pd
import requests
import json
import sherlock
import regras_politica as rp

### Testes Modulares - Referente ao tópico 3/4 - 3.6/4.4 Marcação de Aprovados

### Geração de Massa

In [ ]:
#Gerando a massa modelo passando o caminho do JSON como nome da variável
base_entrada = pd.DataFrame({'solicitante.fonteRenda':["01. Fopag"]
                            })
                            

In [ ]:
#Criação das massas com casos forçados
#Digito CPF Concomitante CP
massa_flagRenda = sherlock.Gera_Massa(base_entrada)
massa_flagRenda.gera_cenario('solicitante.fonteRenda',["02. Funcionários Não FOPAG (J&F)",
                                                                           "04. Renda OPF Confirmada",
                                                                           "08. Blend Externo",
                                                                           "06. Servidor Público",
                                                                           "03. Portabilidade",
                                                                           "07. Blend Interno",
                                                                           "99. Não encontrada",
                                                                           "01. Fopag",
                                                                           "-99999",
                                                                           "00. Dominio X Nao Previsto"])
massa_flagRenda.set_bom_arqv()
massa_flagRenda.converte_massa()

In [ ]:
massa_flagRenda.get_massa_final()

### Execução Modular e Geração de CSV

In [ ]:
listaResultado = rp.execucaoModular(massa_flagRenda.get_massa_json(),rp.DefineFlagRenda)

##### Valida se todas condições estão sendo abordadas

In [ ]:
import pandas as pd
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['saidas']['flagTipoRenda'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

In [ ]:
# Renomeação das colunas para o cabeçalho do RMA
massa_flagRenda.rename_cabecalho()

In [ ]:
massa_flagRenda.atribui_expected(listaResultado,"payloadHomol.saidas.flagTipoRenda","flagTipoRenda")
massa_flagRenda.get_massa_dm()

### Conversão da massa para CSV

In [ ]:
massa_flagRenda.get_massa_dm().to_csv('C:/Users/T51657/Desktop/Homologação/Modelos Online/v2/massas/Modulares/massaFlagTipoRenda.csv',index=False)